In [1]:
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np

class WheatGBSDataSet(Dataset):
    def __init__(self, dbPath, phenoName):
        super().__init__()
        accessionNAME_LIST, self.phenotype_series = self.read_pheno(dbPath  + '/phenotype/phenotype.txt', phenoName)
        self.geno_df = self.read_geno(dbPath  + '/genotype/Genotyping_data_1.hmp.txt', accessionNAME_LIST)

    def __len__(self):
        return len(self.phenotype_series)

    def __getitem__(self, idx):
        genotype  =  torch.tensor(self.geno_df.iloc[idx].values  , dtype=torch.float).unsqueeze(0)
        phenotype =  torch.tensor(self.phenotype_series.iloc[idx], dtype=torch.float)
        return genotype, phenotype
    def read_pheno(self, fileName, phenoName):
        fin_pheno = open(fileName)
        # Read the first line (header)
        legend_LIST = fin_pheno.readline().rstrip('\n').split('\t')

        phenoIDX = legend_LIST.index(phenoName)

        accessionName_LIST = []
        phenoValue_LIST = []

        # Read each line and extract phenotype values (excluding 'NA')
        for line in fin_pheno:
            data_LIST = line.rstrip('\n').split('\t')
            phenoValue = data_LIST[phenoIDX]

            if phenoValue == 'NA': continue
            accessionName = data_LIST[0]
            accessionName_LIST += [accessionName]

            phenoValue = float(phenoValue)
            phenoValue_LIST += [phenoValue]
        fin_pheno.close()

        # Convert phenotype_values to pandas Series
        phenotype_series = pd.Series(phenoValue_LIST, index=accessionName_LIST)

        return accessionName_LIST, phenotype_series
    
    def read_geno(self, fileName, accessionName_LIST):
        # Define IUPAC codes dictionary
        IUPAC_DICT = {
            ('A', 'T'): 'W', ('T', 'A'): 'W', ('C', 'G'): 'S', ('G', 'C'): 'S', ('A', 'C'): 'M', ('C', 'A'): 'M',
            ('G', 'T'): 'K', ('T', 'G'): 'K', ('A', 'G'): 'R', ('G', 'A'): 'R', ('C', 'T'): 'Y', ('T', 'C'): 'Y'
        }

        # Count the total number of SNPs
        snp_size = 0
        fin_geno = open(fileName)
        legend_LIST = fin_geno.readline().rstrip('\n').split('\t')
        for line in fin_geno:
            snp_size += 1
        fin_geno.close()

        accession_size = len(accessionName_LIST)

        # Create an empty DataFrame
        print('accession_size:', accession_size, 'snp_size:', snp_size)
        #geno_df = pd.DataFrame(0, index=accessionName_LIST, columns=range(snp_size * 4))
        geno_array = np.zeros((accession_size, snp_size * 4), dtype=int)

        # Get indices for accession names in the genotype file headers
        accessionIDX_LIST = []
        for accessionName in accessionName_LIST:
            accessionIDX = legend_LIST.index(accessionName)
            accessionIDX_LIST += [accessionIDX]

        # Read genotype information for each SNP
        fin_geno = open(fileName)
        _tmp = fin_geno.readline().rstrip('\n').split('\t')
        for snpIDX, line in enumerate(fin_geno):
            #if snpIDX%10000 == 9999:
            #    print(snpIDX + 1)
            data_LIST = line.rstrip('\n').split('\t')

            homo1, homo2 = data_LIST[1].split('/')
            hetero = IUPAC_DICT[(homo1, homo2)]

            genotype_LIST = [data_LIST[accessionIDX] for accessionIDX in accessionIDX_LIST]

            for accessionIDX, genotype in enumerate(genotype_LIST):
                if genotype == homo1:
                    geno_array[accessionIDX, snpIDX*4 + 0] = 1
                elif genotype == hetero:
                    geno_array[accessionIDX, snpIDX*4 + 1] = 1
                elif genotype == homo2:
                    geno_array[accessionIDX, snpIDX*4 + 2] = 1
                elif genotype == 'N':
                    geno_array[accessionIDX, snpIDX*4 + 3] = 1
                else:
                    print('Unexpected genotype:', genotype)

        geno_df = pd.DataFrame(geno_array, index=accessionName_LIST)
        return geno_df
            
wheatGBS = WheatGBSDataSet('../data/WheatGBS', 'Grain_color')
print(wheatGBS[0][0].shape, wheatGBS[0][1].shape)

accession_size: 1944 snp_size: 78606
torch.Size([1, 314424]) torch.Size([])


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(314424, 65536)
        self.fc2 = nn.Linear(65536, 4096)
        self.fc3 = nn.Linear(4096, 1)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.softmax(x)
        return x

In [3]:
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
import numpy as np
kf = KFold(n_splits=5)
pearson_correlations = []

for train_index, val_index in kf.split(wheatGBS):
    train_dataset = Subset(wheatGBS, train_index)
    val_dataset = Subset(wheatGBS, val_index)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = MLP()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    # 모델 학습
    epoch_losses = []
    for epoch in range(20):  # 예시: 10 에포크 동안 학습
        model.train()
        batch_losses = []
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            batch_losses.append(loss.item())
            
            loss.backward()
            optimizer.step()

        epoch_loss = np.mean(batch_losses)
        epoch_losses.append(epoch_loss)
        print(f'Train Epoch: {epoch} \tLoss: {epoch_loss:.6f}')
        
    # 모델 검증
    model.eval()
    val_targets = []
    val_predictions = []
    test_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            test_loss += criterion(outputs, targets)
            val_targets.extend(targets.numpy())
            val_predictions.extend(outputs.numpy())

    val_targets = np.array(val_targets)
    val_predictions = np.array(val_predictions)

    #print(val_targets.flatten())
    #print(val_predictions.flatten())
    #print('\n')

    pearson_corr = np.corrcoef(val_targets.flatten(), val_predictions.flatten())[0, 1]
    pearson_correlations.append(pearson_corr)

    print(f'Test set: loss: {test_loss:.4f}, Accuracy: {pearson_corr:.4f}\n')

average_pearson_corr = np.mean(pearson_correlations)
print("Average Pearson Correlation:", average_pearson_corr)

RuntimeError: Expected target size [32, 1], got [32]

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(wheatGS_data.geno_df, wheatGS_data.phenotype_series, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVR
svm_model = SVR(kernel='rbf', verbose=True)

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
svm_model.score(X_test, y_test)